In [2]:
import numpy as np
import pandas as pd
# from icecream import ic
import subprocess
import json
import os
import plotly.graph_objects as go
from matplotlib import pyplot as plt
# import geopandas as gpd
# from shapely.geometry import Polygon
# from shapely.ops import cascaded_union, unary_union
# from shapely import geometry
# from geopy import distance
%matplotlib inline
import plotly.express as px 
from scipy import signal
from tqdm import tqdm
# from Bio import Phylo
# import ete3
import itertools
import sklearn

In [19]:
variants = pd.read_csv('variants_full.csv')


In [8]:
cryptic = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/CRyPTIC_reuse_table_20221019.csv')

In [22]:
overlap = set(variants['sample_id']).intersection(set(cryptic['wgs_id'].to_list()))

In [23]:
variants = variants[variants['sample_id'].isin(overlap)]

In [45]:
variants['sample_id'].unique().shape

(11920,)

In [25]:
variants['SNP'] = variants['gene'] + '-'+ variants['change']

In [27]:
# variants_unique = variants.drop_duplicates(subset='SNP')

In [32]:
def compare_snp_lists(set_list, query_list):
    # Convert query_list to a set for O(1) lookup time
    query_set = query_list

    # Use numpy for efficient array operations
    import numpy as np
    # Convert set_list to a NumPy array for vectorized operations
    set_array = np.array(set_list)
    # Create an array of zeros with the same length as set_list
    output_array = np.zeros(len(set_list), dtype=int)
    # Update positions in output_array to 1 where elements of set_array are in query_set
    output_array[np.isin(set_array, query_set)] = 1

    return output_array.tolist()

# Example usage
set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
query_list = ['SNP2', 'SNP4']
output_list = compare_snp_lists(set_list, query_list)
print(output_list)  # Expected Output: [0, 1, 0, 1]


[0, 1, 0, 1]


In [67]:
def compare_snp_lists_with_values(set_list, query_list, values_list):
    # Create a dictionary from query_list and values_list for faster lookups
    query_dict = dict(zip(query_list, values_list))
    
    # Initialize output list with zeros
    output_list = [0] * len(set_list)
    
    # Iterate through the set list and update the output list
    for i, snp in enumerate(set_list):
        if snp in query_dict:
            # Use the corresponding value from values_list instead of 1
            output_list[i] = query_dict[snp]
    
    return output_list

# Example usage:
set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
query_list = ['SNP2', 'SNP4']
values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
output_list = compare_snp_lists_with_values(set_list, query_list, values_list)
print(output_list)  # Expected output: [0, 5, 0, 10]


def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
    # Create a dictionary from query_list and values_list for direct mapping
    query_dict = dict(zip(query_list, values_list))
    
    # Use list comprehension to build the output list directly
    output_list = [query_dict.get(snp, 0) for snp in set_list]
    
    return output_list

# Example usage
set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
query_list = ['SNP2', 'SNP4']
values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
print(output_list)  # Expected output: [0, 5, 0, 10]


[0, 5, 0, 10]
[0, 5, 0, 10]


In [139]:
len(variants[variants['sample_id']==x]['SNP'].to_list())

46

In [141]:
len(variants[variants['sample_id']==x]['freq'].to_list())

46

In [68]:
aa = []
all_snp = variants['SNP'].unique()
for x in tqdm(variants['sample_id'].unique()):
    aa.append(compare_snp_lists_with_values(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))

100%|██████████| 11920/11920 [11:16<00:00, 17.63it/s]


In [ ]:
variants[variants['sample_id']==x]['SNP'].to_list()

In [ ]:
aa_array = np.array(aa)
np.savetxt('data_aa/aa_data.csv', aa_array, delimiter=',')   # X is an array

In [106]:
cryptic_mic = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data1/CRyPTIC_reuse_table_20221019_cleaned.csv')

In [133]:
variants_ind = variants[variants['gene']=='rpoB']
all_snp = variants_ind['SNP'].unique()

In [136]:
len(all_snp)

869

# Getting rpob aa snp data

In [151]:
all_snp

array(['rpoB-c.309C>T', 'rpoB-c.1482G>A', 'rpoB-p.Ser450Leu',
       'rpoB-p.Glu250Gly', 'rpoB-p.Asp435Val', 'rpoB-p.Ser450Phe',
       'rpoB-p.Thr400Ala', 'rpoB-p.His445Tyr', 'rpoB-c.-61C>T',
       'rpoB-p.Val359Ala', 'rpoB-c.1683G>A', 'rpoB-c.684C>T',
       'rpoB-p.Ile751Val', 'rpoB-p.Lys944Asn', 'rpoB-c.-113C>T',
       'rpoB-p.His445Leu', 'rpoB-p.His445Asp', 'rpoB-p.Lys891Glu',
       'rpoB-p.Asp435Tyr', 'rpoB-p.Gln432Lys', 'rpoB-p.Glu761Asp',
       'rpoB-p.Val695Leu', 'rpoB-p.His445Arg', 'rpoB-p.Arg827Leu',
       'rpoB-c.1299C>T', 'rpoB-p.Ala286Val', 'rpoB-c.1297_1299dupTTC',
       'rpoB-p.Leu449Met', 'rpoB-p.Leu554Pro', 'rpoB-c.138A>C',
       'rpoB-p.Leu430Pro', 'rpoB-p.Arg552His', 'rpoB-c.1292_1294dupGCC',
       'rpoB-p.Asp634Gly', 'rpoB-c.1410C>T', 'rpoB-c.-49C>T',
       'rpoB-p.Ile488Val', 'rpoB-p.Asp531Glu', 'rpoB-p.Thr399Ile',
       'rpoB-p.Pro768Leu', 'rpoB-p.Leu452Pro', 'rpoB-c.2025C>A',
       'rpoB-p.His445Asn', 'rpoB-p.Arg219Cys', 'rpoB-c.1659G>A',
       'rpoB

In [84]:
variants_ind = variants[variants['gene']=='rpoB']
aa_rpoB = []
all_snp = variants_ind['SNP'].unique()

for x in tqdm(variants['sample_id'].unique()):
    if x in variants_ind['sample_id'].unique():
        aa_rpoB.append(compare_snp_lists_with_values(all_snp, variants_ind[variants_ind['sample_id']==x]['SNP'].to_list(), variants_ind[variants_ind['sample_id']==x]['freq'].to_list()))
    else:
        aa_rpoB.append([0]*len(all_snp))
aa_rpoB_array = np.array(aa_rpoB)
np.savetxt('data_aa/aa_rpoB.csv', aa_rpoB_array, delimiter=',')   # X is an array

100%|██████████| 11920/11920 [00:31<00:00, 373.60it/s]


In [131]:
np.savetxt('data_aa/train_aa_rpoB.csv', aa_rpoB_array[:int(0.9*aa_rpoB_array.shape[0])], delimiter=',')   # X 
np.savetxt('data_aa/test_aa_rpoB.csv', aa_rpoB_array[int(0.9*aa_rpoB_array.shape[0]):], delimiter=',')   # X 

In [ ]:
np.savetxt('data_aa/aa_rpoB.csv', aa_rpoB_array, delimiter=',')   # X is an array

# Getting eth aa snp data

In [87]:
variants_ind = variants[variants['gene'].isin(['embA','embB','embC'])]
aa_emb = []
all_snp = variants_ind['SNP'].unique()

for x in tqdm(variants['sample_id'].unique()):
    if x in variants_ind['sample_id'].unique():
        aa_emb.append(compare_snp_lists_with_values(all_snp, variants_ind[variants_ind['sample_id']==x]['SNP'].to_list(), variants_ind[variants_ind['sample_id']==x]['freq'].to_list()))
    else:
        aa_emb.append([0]*len(all_snp))
aa_emb_array = np.array(aa_emb)
np.savetxt('data_aa/aa_emb.csv', aa_emb_array, delimiter=',')   # X is an array

100%|██████████| 11920/11920 [02:10<00:00, 91.03it/s]


In [132]:
np.savetxt('data_aa/aa_emb.csv', aa_emb_array, delimiter=',')   # X is an array
np.savetxt('data_aa/train_aa_emb.csv', aa_emb_array[:int(0.9*aa_emb_array.shape[0])], delimiter=',')   # X 
np.savetxt('data_aa/test_aa_emb.csv', aa_emb_array[int(0.9*aa_emb_array.shape[0]):], delimiter=',')   # X 

# Getting corresponding mic snp dataset for the above

In [145]:
cryptic_mic['wgs_id'] = pd.Categorical(cryptic_mic['wgs_id'], categories=variants['sample_id'].unique().tolist(), ordered=True)
# Sort the DataFrame by the 'wgs_id' column

cryptic_mic_ordered = cryptic_mic.sort_values('wgs_id')
overlap = set(variants['sample_id']).intersection(set(cryptic['wgs_id'].to_list()))
cryptic_mic_ordered = cryptic_mic_ordered[cryptic_mic_ordered['wgs_id'].isin(overlap)]

In [146]:
np.savetxt('data_aa/ETH_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['ETH_MIC']]), delimiter=',')   # X is an array
np.savetxt('data_aa/train_ETH_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['ETH_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])], delimiter=',')   # X is an array
np.savetxt('data_aa/test_ETH_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['ETH_MIC']])[int(0.9*cryptic_mic_ordered.shape[0]):], delimiter=',')   # X is an array

np.savetxt('data_aa/RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']]), delimiter=',')   # X is an array
np.savetxt('data_aa/train_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])], delimiter=',')   # X is an array
np.savetxt('data_aa/test_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[int(0.9*cryptic_mic_ordered.shape[0]):], delimiter=',')   # X is an array

In [92]:
cryptic_mic['ETH_MIC']

Index(['UNIQUEID', 'AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE', 'AMI_MIC', 'BDQ_MIC',
       'CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC',
       'AMI_PHENOTYPE_QUALITY', 'BDQ_PHENOTYPE_QUALITY',
       'CFZ_PHENOTYPE_QUALITY', 'EMB_PHENOTYPE_QUALITY',
       'ETH_PHENOTYPE_QUALITY', 'INH_PHENOTYPE_QUALITY',
       'KAN_PHENOTYPE_QUALITY', 'LEV_PHENOTYPE_QUALITY',
       'LZD_PHENOTYPE_QUALITY', 'MXF_PHENOTYPE_QUALITY',
       'RIF_PHENOTYPE_QUALITY', 'RFB_PHENOTYPE_QUALITY', 'ENA_SAMPLE', 'VCF',
       'REGENOTYPED_VCF', 'wgs_id'],
      dtype='object')

In [ ]:
from tqdm import tqdm
import numpy as np

def convert_to_coordinate_value(input_list):
    return [(i, val) for i, val in enumerate(input_list) if val != 0]

# Generate coordinate values
aa_coord = [convert_to_coordinate_value(x) for x in tqdm(aa_data_freq)]

# Find the length of the longest list
max_length = max(len(item) for item in aa_coord)

# Pad the lists to have uniform length
padded_aa_coord = [item + [(0, 0)] * (max_length - len(item)) for item in aa_coord]

# Convert to a high-dimensional NumPy array
aa_coord_array = np.array(padded_aa_coord)

# The result is a high-dimensional NumPy array
print(aa_coord_array.shape)


In [ ]:
from tqdm import tqdm
import numpy as np

def convert_to_coordinate_value(input_list):
    return [(i, val) for i, val in enumerate(input_list) if val != 0]

# Generate coordinate values
aa_coord = [convert_to_coordinate_value(x) for x in tqdm(aa_data_freq)]

# Find the length of the longest list
max_length = max(len(item) for item in aa_coord)

# Pad the lists to have uniform length
padded_aa_coord = [item + [(0, 0)] * (max_length - len(item)) for item in aa_coord]

# Convert to a high-dimensional NumPy array
aa_coord_array = np.array(padded_aa_coord)

# The result is a high-dimensional NumPy array
print(aa_coord_array.shape)


In [ ]:
for x in np.unique(variants['sample_id']):
     print(variants[variants['sample_id'] == 'ERR9787047'].shape)

In [7]:
variants['change'].value_counts()

n.-187C>T               135403
p.Glu21Gln              133939
c.33A>G                 133602
c.-165T>C               133108
p.Ile948Val             133064
                         ...  
c.-4164_*1837del             1
p.Ala876Ser                  1
c.2192_2193insGCCTAG         1
c.536_537insGCCCG            1
n.1011T>G                    1
Name: change, Length: 109920, dtype: int64